# Here we analayse the Results of ChatGPT for the input with just annotations

In [1]:
from support_functions.load_json_chatgpt_results import load_chat_gpt_results
from support_functions.excel_helper import prepaire_excel_data

import pandas as pd

## Load ChatGPT results

In [2]:
directories_to_ignore:list = ["text-annoations-redundancy-model-gpt-3.5-turbo", "text-annotations-redundancy-model-gpt-4o", "annoations-redundancy-model-gpt-4o"]
chat_gpt_results: dict[dict[str, list[dict]]] = load_chat_gpt_results(directories_to_ignore)
chat_gpt_result_keys: list[str] = list(chat_gpt_results.keys())
chat_gpt_result_json_keys = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in chat_gpt_results.items()
}

## Analyse the data sets

In [3]:
model: str
data_set: str
model = data_set = None
full_main_part: int 
partial_main_part: int
full_benefit: int
partial_benefit: int
partial_main_part = full_main_part = partial_benefit = full_benefit = 0

In [4]:
for key, item in chat_gpt_results.items():
    model = key
    for sub_key, sub_item in item.items():
        data_set = sub_key
        for _ in sub_item:
            if bool(_["mainPartRedundancies"]["partialRedundancy"]):
                partial_main_part += 1
            elif bool(_["mainPartRedundancies"]["fullRedundancy"]):
                full_main_part += 1
            if bool(_["benefitRedundancies"]["partialRedundancy"]):
                partial_benefit += 1
            elif bool(_["benefitRedundancies"]["fullRedundancy"]):
                full_benefit += 1
        print(f"{model}, {data_set}, {partial_main_part}, {full_main_part}, {partial_benefit}, {full_benefit}")
        full_main_part = partial_benefit = full_benefit = partial_benefit = 0

annoations-redundancy-model-gpt-3.5-turbo, 00_g03.json, 0, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g04.json, 0, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g08.json, 1, 0, 1, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g10.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g11.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g14.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g16.json, 1, 0, 1, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g18.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g19.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g21.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g22.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g23.json, 1, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g24.json, 2, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g25.json, 2, 0, 0, 0
annoations-redundancy-model-gpt-3.5-turbo, 00_g26.json, 2, 0, 

### Load Graph Data

In [5]:
excel_data_relative_ground_truth = prepaire_excel_data()
display(excel_data_relative_ground_truth.head(3))
#print(str(excel_data_relative_ground_truth.to_csv(header=False)).replace("_x000D_", "").replace("\n\n", " ")) 

,Project Number,Redundant Pairs,User Stories Texts,Total Redundancy Clause *,Corresponding USID 1,Corresponding USID 2,Main Part Redundancy Clause**,Benefit Part Redundancy Clause ***,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,G03,user_story_12_AND_user_story_39,"user_story_12: #g03# as a staff member, i want...",1,326,353,0,1,False,False,True,False
2,G03,user_story_17_AND_user_story_30,"user_story_17: #g03# as a staff member, i want...",1,331,344,0,1,False,False,True,False
3,G04,user_story_05_AND_user_story_12,"user_story_05: #g04# as a user, i want to have...",1,168,175,0,1,False,False,False,True


In [6]:
excel_data_graph_approach = prepaire_excel_data("Framework")
display(excel_data_graph_approach.head(3))
#print(str(excel_data_graph_approach.to_csv(header=False)).replace("_x000D_", "").replace("\n\n", " ")) 

,Project Number,Redundant Pairs,User Stories Texts,Total Redundancy Clause *,Corresponding USID 1,Corresponding USID 2,Main Part Redundancy Clause**,Benefit Part Redundancy Clause ***,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,g03,user_story_12_AND_user_story_39,"user_story_12: #g03# as a staff member, i want...",1,326,353,0,1,False,False,True,False
2,g03,user_story_17_AND_user_story_30,"user_story_17: #g03# as a staff member, i want...",1,331,344,0,1,False,False,False,True
3,g04,user_story_05_AND_user_story_12,"user_story_05: #g04# as a user, i want to have...",1,168,175,0,1,False,False,False,True


## Compare to Graph Data

In [7]:
def determine_diff(json_data: dict[list[dict]],
                   keys_for_eval: list,
                   excel_data: pd.DataFrame,
                   reference_to_redundancy_part_json: str,
                   partial_no_difference: list[dict],
                   full_no_difference: list[dict],
                   partial_difference_predicted_false: list[dict],
                   full_difference_predicted_false: list[dict],
                   partial_difference_predicted_true: list[dict],
                   full_difference_predicted_true: list[dict],
                   redundancy_detected: list[dict],
                   entries_in_filtered_row: int = -1,
                   usid1: int = -1,
                   usid2: int = -1,
                   ENTRY_IN_GROUND_TRUTH: int = 1,
                   ENTRY_NOT_IN_GROUND_TRUTH: int = 0) -> None:
    PARTIAL_REDUNDANCY = "partialRedundancy"
    FULL_REDUNDANCY = "fullRedundancy"
   
    # We assume here that the .json-Files have always all posible pairs n(n-1)/2
    idx_np_array: int = 0
    predicted: bool = False
    graph_prediction: bool = False
    already_assigned: bool = False
    pair: dict = None
    
    for v in json_data.values():
        for key in v.keys():
            json_item = v[key]
            for item in json_item:
                try:
                    usid1, usid2 = item[keys_for_eval[0]]
                    pair = {
                        "PID": str(key).replace(".json", ""),
                        "USID1": usid1,
                        "USID2": usid2
                    }
                except (ValueError, KeyError) as e:
                    print(f"Error extracting UIDs from item: {item}, key: {key}")
                    raise e
                filtered_rows = excel_data[(excel_data['Corresponding USID 1'].astype(int) == int(usid1)) 
                                        & (excel_data['Corresponding USID 2'].astype(int) == int(usid2))]
                entries_in_filtered_row = len(filtered_rows)
                if  entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                            
                # The Excel does not contain false positives and true negatives and false negatives
                # It just contains true positives "mainPartRedundancies"
                if (entries_in_filtered_row == ENTRY_IN_GROUND_TRUTH):
                    # Firstly, analyse main part and if it is just wrong labelled
                    graph_prediction = bool(filtered_rows.iat[0, keys_for_eval[4]] >= 1)
                    predicted = bool(item[reference_to_redundancy_part_json][FULL_REDUNDANCY])
                    if ((graph_prediction == True and predicted == True)
                            or graph_prediction == False and predicted == False):
                        full_no_difference.append({**pair, "Redundancy": graph_prediction})
                    elif graph_prediction == True and predicted == False:
                        full_difference_predicted_false.append(pair)
                    elif graph_prediction == False and predicted == True:
                        full_difference_predicted_true.append(pair)                 
                    else:
                        raise UnboundLocalError("Not assignable")
                    
                    # Secondly, analyse benefit and if it is just wrong labelled
                    graph_prediction = bool(filtered_rows.iat[0, keys_for_eval[5]] >= 1)
                    predicted = bool(item[reference_to_redundancy_part_json][PARTIAL_REDUNDANCY])
                    if ((graph_prediction == True and predicted == True)
                        or graph_prediction == False and predicted == False):
                        partial_no_difference.append({**pair, "Redundancy": graph_prediction})
                    elif graph_prediction == True and predicted == False:
                        partial_difference_predicted_false.append(pair)
                    elif graph_prediction == False and predicted == True:
                        partial_difference_predicted_true.append(pair)                 
                    else:
                        raise UnboundLocalError("Not assignable")
                    
                    # Thirdly, if an redundancy is detected but wrong labeled
                    already_assigned = (
                        bool(filtered_rows.iat[0, keys_for_eval[4]] >= 1) == bool(item[reference_to_redundancy_part_json][FULL_REDUNDANCY])
                        or bool(filtered_rows.iat[0, keys_for_eval[5]] >= 1) == bool(item[reference_to_redundancy_part_json][PARTIAL_REDUNDANCY])
                    )
                    graph_prediction = bool(filtered_rows.iat[0, keys_for_eval[3]] >= 1)
                    if (((graph_prediction == bool(item[reference_to_redundancy_part_json][FULL_REDUNDANCY]))
                            or (graph_prediction == bool(item[reference_to_redundancy_part_json][PARTIAL_REDUNDANCY])))
                            and not already_assigned):
                        redundancy_detected.append(pair)               
                elif(entries_in_filtered_row == ENTRY_NOT_IN_GROUND_TRUTH):
                    graph_prediction = False
                    
                    # Firstly, analyse main part and if it is just wrong labelled
                    predicted = bool(item[reference_to_redundancy_part_json][FULL_REDUNDANCY])
                    if ((graph_prediction == True and predicted == True)
                            or graph_prediction == False and predicted == False):
                        full_no_difference.append({**pair, "Redundancy": graph_prediction})
                    elif graph_prediction == True and predicted == False:
                        full_difference_predicted_false.append(pair)
                    elif graph_prediction == False and predicted == True:
                        full_difference_predicted_true.append(pair)                 
                    else:
                        raise UnboundLocalError("Not assignable")
                    
                    # Secondly, analyse benefit and if it is just wrong labelled
                    predicted = bool(item[reference_to_redundancy_part_json][PARTIAL_REDUNDANCY])
                    if ((graph_prediction == True and predicted == True)
                        or graph_prediction == False and predicted == False):
                        partial_no_difference.append({**pair, "Redundancy": graph_prediction})
                    elif graph_prediction == True and predicted == False:
                        partial_difference_predicted_false.append(pair)
                    elif graph_prediction == False and predicted == True:
                        partial_difference_predicted_true.append(pair)                 
                    else:
                        raise UnboundLocalError("Not assignable")
                else:
                    raise UnboundLocalError("Not assignable: Totally.")
                idx_np_array += 1
                
                predicted = graph_prediction = False
                entries_in_filtered_row = usid1 = usid2 = -1
                pair = None

In [8]:
def build_csv(data_list: list[dict], first_column: str) -> str:
    separator: str = ";"
    temp = ''.join(
        first_column + separator + ''.join(f"{value}{separator}" for key, value in item.items()) + "\n"
        for item in data_list
    )
    return temp.rstrip("\n")

In [9]:

def calc_redundancy(excel_data: pd.DataFrame, json_data: dict[list[dict]]) -> None:
    main_part_partial_no_difference: list[dict] = []
    main_part_full_no_difference: list[dict] = []
    benefit_partial_no_difference: list[dict] = []
    benefit_full_no_difference: list[dict] = []
    # The graph approach determined a redundancy but not ChatGPT
    main_part_partial_difference_predicted_false: list[dict] = []
    main_part_full_difference_predicted_false: list[dict] = []
    benefit_partial_difference_predicted_false: list[dict] = []
    benefit_full_difference_predicted_false: list[dict] = []
    # ChatGPT determined a redundancy but not the graph approach
    main_part_partial_difference_predicted_true: list[dict] = []
    main_part_full_difference_predicted_true: list[dict] = []
    benefit_partial_difference_predicted_true: list[dict] = []
    benefit_full_difference_predicted_true: list[dict] = []
    # Redundancy detected but wrong labelled
    main_part_redundancy_detected: list[dict] = []
    benefit_redundancy_detected: list[dict] = []
    
    base_keys = [
        'relatedStories',
    ]
    
    keys_for_eval = base_keys + [
        'redundantMainPart',
        'mainPartRedundancies',
        6,
        8,
        9,
    ]
    
    determine_diff(
        json_data=json_data,
        keys_for_eval=keys_for_eval,
        excel_data=excel_data_graph_approach,
        reference_to_redundancy_part_json="mainPartRedundancies",
        partial_no_difference=main_part_partial_no_difference,
        full_no_difference=main_part_full_no_difference,
        partial_difference_predicted_true=main_part_partial_difference_predicted_true,
        full_difference_predicted_true=main_part_full_difference_predicted_true,
        partial_difference_predicted_false=main_part_partial_difference_predicted_false,
        full_difference_predicted_false=main_part_full_difference_predicted_false, 
        redundancy_detected=main_part_redundancy_detected
    )
    
    print(build_csv(main_part_partial_no_difference, "Main Part - Partial: No Difference"))
    print(build_csv(main_part_full_no_difference, "Main Part - Full: No Difference"))
    print(build_csv(main_part_partial_difference_predicted_false, "Main Part - Partial: Predicted false negative"))
    print(build_csv(main_part_full_difference_predicted_false, "Main Part - Full: false negative"))
    print(build_csv(main_part_partial_difference_predicted_true, "Main Part - Partial: false positive"))
    print(build_csv(main_part_full_difference_predicted_true, "Main Part - Full: false positive"))
    print(build_csv(main_part_redundancy_detected, "Main Part - Redundancy detected but not correctly mapped"))
    
    keys_for_eval = base_keys +  [
        'redundantBenefit',
        'benefitRedundancies',
        7,
        10,
        11,
    ]
    
    determine_diff(
        json_data=json_data,
        keys_for_eval=keys_for_eval,
        excel_data=excel_data_graph_approach,
        reference_to_redundancy_part_json="benefitRedundancies",
        partial_no_difference=benefit_partial_no_difference,
        full_no_difference=benefit_full_no_difference,
        partial_difference_predicted_true=benefit_partial_difference_predicted_true,
        full_difference_predicted_true=benefit_full_difference_predicted_true,
        partial_difference_predicted_false=benefit_partial_difference_predicted_false,
        full_difference_predicted_false=benefit_full_difference_predicted_false, 
        redundancy_detected=benefit_redundancy_detected
    )
    
    print(build_csv(benefit_partial_no_difference, "Benefit - Partial: No Difference"))
    print(build_csv(benefit_full_no_difference, "Benefit - Full: No Difference"))
    print(build_csv(benefit_partial_difference_predicted_false, "Benefit - Partial: Predicted false negative"))
    print(build_csv(benefit_full_difference_predicted_false, "Benefit - Full: Predicted false negative"))
    print(build_csv(benefit_partial_difference_predicted_true, "Benefit - Partial: Predicted false positive"))
    print(build_csv(benefit_full_difference_predicted_true, "Benefit - Full: Predicted false positive"))
    print(build_csv(benefit_redundancy_detected, "Benefit - Redundancy detected but not correctly mapped"))

In [10]:
calc_redundancy(
    excel_data=excel_data_graph_approach,
    json_data=chat_gpt_results
)

Main Part - Partial: No Difference;00_g03;315;316;False;
Main Part - Partial: No Difference;00_g03;315;317;False;
Main Part - Partial: No Difference;00_g03;315;318;False;
Main Part - Partial: No Difference;00_g03;315;319;False;
Main Part - Partial: No Difference;00_g03;315;320;False;
Main Part - Partial: No Difference;00_g04;164;166;False;
Main Part - Partial: No Difference;00_g04;164;167;False;
Main Part - Partial: No Difference;00_g04;164;168;False;
Main Part - Partial: No Difference;00_g04;164;169;False;
Main Part - Partial: No Difference;00_g08;426;428;False;
Main Part - Partial: No Difference;00_g08;426;430;False;
Main Part - Partial: No Difference;00_g10;493;494;False;
Main Part - Partial: No Difference;00_g10;493;495;False;
Main Part - Partial: No Difference;00_g10;493;496;False;
Main Part - Partial: No Difference;00_g10;493;497;False;
Main Part - Partial: No Difference;00_g10;493;498;False;
Main Part - Partial: No Difference;00_g11;590;591;False;
Main Part - Partial: No Differe